In [ ]:
%matplotlib inline
from __future__ import division
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import struct
import pickle
import pandas as pd
import random

In [ ]:
import sys
sys.path.append('../../src')

In [ ]:
from pylab import rcParams
from IPython.display import clear_output
rcParams['figure.figsize'] = 16, 8
pd.options.display.max_columns = 999

In [ ]:
from utils import FloatConvert32, FloatConvert64
from microssembly2 import Microssembly
from price_generator import random_walk, random_price

In [ ]:
def get_strategy_signal(price, short_ma, long_ma, demo=False):
    short_ma = abs(int(short_ma))
    long_ma = abs(int(long_ma))
    shares = 1

    try:
        ms = price.rolling(short_ma).mean()
        ml = price.rolling(long_ma).mean()

        buy = (ms > ml) & (ms.shift() < ml.shift())  # ma cross up
        sell = (ms < ml) & (ms.shift() > ml.shift())  # ma cross down
        signal = pd.Series(index=price.index)
        signal[buy[buy != False].index] = shares
        signal[sell[sell != False].index] = 0
    except Exception as e:
        print('Long {} Short {}'.format(long_ma, short_ma))
        raise e

    if demo is True:
        ax = price.plot()
        ms.plot()
        ml.plot()
        plt.legend(['price', 'Short MA', 'Long MA'], loc='best')
        ax.xaxis.grid(True, which='minor', linestyle='-', linewidth=0.25)

    return signal

def get_strategy_signal_1(price):
    return get_strategy_signal(price, 2, 8, False)

In [ ]:
price = random_price()
signal = get_strategy_signal(price, 2, 8, True)
price.shape, signal.dropna().shape, signal.dropna()[:5]

In [ ]:
from simple_strategy import get_signal, loss_function, eval_individual, find_best_model

In [ ]:
mssembly = Microssembly(architecture = 4)
data = [1, 2, 3, 4]
mssembly.load_data(data)
mssembly.memory

In [ ]:
prog = ([
    '110000001000',
    '001110001110',
    '110000011000',
    '001110001110',
    '110000101000',
    '001110001110',
    '110000111000',
    '001110001110',
    '101011000000',
    '101011000000',
    '101011000000',
    '101011000000',
    '010011101100',
    '110011001111' 
])
len(''.join(prog))

In [ ]:
mssembly.run(''.join(prog), cycles=100)

In [ ]:
assert mssembly.memory[15] == np.average(data), "Something went wrong!!!"

In [ ]:
loss = []
for _ in range(1000):
    price = random_price()
    random_code = ''.join(list(map(str, np.random.randint(0, 2, size=1000))))
    signal = get_signal(price, random_code)
    observed_signal = get_strategy_signal_1(price)
    l = loss_function(signal, observed_signal)    
    if(l > 0):
        loss.append(l)
print(np.average(loss))

In [ ]:
pop, hof, _ = find_best_model(get_strategy_signal_1,
                              ngen=100,
                              pop_size=1000,
                              mutpb=0.001,
                              indpb=0.1)

In [ ]:
best_code = ''.join(map(str, hof[0]))

In [ ]:
all_loss = []
for i in range(100):
    price = random_price()
    signal = get_signal(price, best_code)
    observed_signal = get_strategy_signal_1(price)
    all_loss.append(loss_function(signal, observed_signal))
np.average(all_loss)

In [ ]:
signal.dropna()[:10]

In [ ]:
pd.DataFrame({'true': observed_signal, 'generated': signal}).dropna().plot.bar()

In [ ]:
import microssembly
mssembly = Microssembly(in_memory_length=3, out_memory_length=1, registers_length=3)
mssembly.out_memory[0] = np.NaN
mssembly.run(best_code)

In [ ]:
a = sum(signal[observed_signal[observed_signal == 1].index] == 1) / len(observed_signal[observed_signal == 1]) + 1e-9
b = sum(signal[observed_signal[observed_signal == 0].index] == 0) / len(observed_signal[observed_signal == 0]) + 1e-9
c = 1 - np.abs(len(observed_signal.dropna()) - len(signal.dropna())) / max(len(observed_signal.dropna()), len(signal.dropna())) + 1e-9

In [ ]:
3 / (1/a + 1/b + 1/c)

In [ ]:
a = 0.9
b = 0.9
c = 0.3
3 / (1/a + 1/b + 1/c)

In [ ]:
d = pd.read_csv('/Users/vorl/Downloads/allstocks.csv')

In [ ]:
d[(d.buy == 1) & (d.sell == 1)]

In [ ]:
d.shape

In [ ]:
d[(d.buy == 1) | (d.sell == 1)].shape

In [ ]:
2935 / 101673

In [ ]:
60 * 0.028

In [ ]:
1000 * 0.028